In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""
import logging
logging.basicConfig(level=logging.INFO)
from transformers import Wav2Vec2FeatureExtractor
from transformers import AutoModel
from mert_fairseq.models.mert import MERTConfig, MERTModel
from mert_fairseq.tasks.mert_pretraining import MERTPretrainingTask
import torch
import yaml
from torch import nn
import torchaudio.transforms as T
import torchaudio
import numpy as np
import inspect
from fairseq import checkpoint_utils, data, options, tasks
from fairseq.data.data_utils import compute_mask_indices
from timm.models.vision_transformer import Block, PatchEmbed
from omegaconf import OmegaConf
import torchaudio
from nnAudio.features.mel import MelSpectrogram

from musiceval.tasks import GTZAN
from mert_fairseq.models.mert import model_cqt_pred, MERTModel, MERTConfig
from mert_fairseq.models.mert.utils import get_1d_sincos_pos_embed
from mert_fairseq.tasks.mert_pretraining import MERTPretrainingTask, MERTPretrainingConfig

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
with open("mert_fairseq/config/pretrain/MERT_RVQ-VAE_CQT_95M.yaml", "r") as f:
    yaml_dict = yaml.safe_load(f)

cfg = OmegaConf.create(yaml_dict)
cfg['model']['label_rate'] = 75.
cfg['model']['mask_type'] = "mae" 
# cfg['model']['audio_extract_type'] = "spec_mlp" 
cfg['model']['audio_mel_bins'] = 128
cfg["task"]["data"] = 'data/fma_medium'
cfg["task"]["label_dir"] = 'data/encodec_labels/fma_medium'
cfg["task"]["labels"] = ["encodec_0"]

model_cfg = MERTConfig(**cfg["model"])
task_cfg = MERTPretrainingConfig(**cfg["task"])
task = MERTPretrainingTask(task_cfg)
model = MERTModel(model_cfg, task_cfg, task.dictionaries)
task.load_dataset("valid")
batch_iter = task.get_batch_iterator(
    dataset=task.dataset("valid"),
    max_tokens=120000*5,
    ignore_invalid_inputs=True,
    num_workers=0,
    epoch=1,
).next_epoch_itr(shuffle=False)
batch = next(batch_iter)
input = batch["net_input"]['source']
padding_mask = batch["net_input"]['padding_mask']
target_list = batch["target_list"]

INFO:mert_fairseq.tasks.mert_pretraining:current directory is /data/home/acw713/workspace/mert/MERT
INFO:mert_fairseq.tasks.mert_pretraining:MERTPretrainingTask Config MERTPretrainingConfig(_name='mert_pretraining', data='data/fma_medium', sharding_data=-1, load_random_data_shard=True, fine_tuning=False, labels=['encodec_0'], label_dir='data/encodec_labels/fma_medium', label_rate=75.0, sample_rate=24000, normalize=False, enable_padding=False, max_keep_size=None, max_sample_size=120000, min_sample_size=72000, single_target=False, random_crop=True, pad_audio=False, store_labels=False, numpy_memmap_label=False, augmentation_effects='[]', augmentation_probs='[]', inbatch_noise_augment_len_range='[8000, 24000]', inbatch_noise_augment_number_range='[1, 3]', inbatch_noise_augment_volume=1.0, dynamic_crops='[]', dynamic_crops_epoches='[]', cqt_loss_bin_dataloader=-1)
INFO:mert_fairseq.models.mert.mert_model:MERTModel Config: MERTConfig(_name='mert', label_rate=75.0, extractor_mode='default', e

CQT kernels created, time used = 1.1683 seconds


/data/home/acw713/.conda/envs/map/lib/python3.8/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
INFO:mert_fairseq.data.mert_dataset:max_keep=None, min_keep=72000, loaded 24, skipped 0 short and 0 long, longest-loaded=720065, shortest-loaded=719438
INFO:mert_fairseq.data.mert_dataset:preparing labels
24it [00:00, 17592.33it/s]
INFO:mert_fairseq.data.mert_dataset:pad_audio=False, random_crop=True, normalize=False, max_sample_size=120000
INFO:fairseq.tasks.fairseq_task:can_reuse_epoch_itr = False
INFO:fairseq.tasks.fairseq_task:reuse_dataloader = True
INFO:fairseq.tasks.fairseq_task:rebuild_batches = False
INFO:fairseq.tasks.fairseq_task:creating new batches for epoch 1


In [3]:
loss_dict = model.forward(input, target_list=target_list, padding_mask=padding_mask, mask=True, features_only=False)
# x = feature_dict['x']
# masked_indices = feature_dict['masked_indices']
# id_restore = feature_dict['id_restore']
# x.shape, masked_indices.shape, id_restore.shape

torch.Size([5, 128, 376])
torch.Size([5, 192, 512])
torch.Size([5, 38, 768]) tensor([5, 4, 4, 3, 4, 5, 4, 4, 3, 3, 5, 3, 5, 4, 2, 4, 4, 5, 4, 5, 5, 3, 3, 5,
        4, 5, 3, 4, 5, 5, 3, 5, 5, 4, 1, 4, 4, 2, 4, 3, 4, 3, 4, 3, 4, 3, 5, 4,
        5, 4, 3, 4, 3, 5, 5, 4, 4, 3, 4, 3, 4, 4, 4, 3, 5, 4, 5, 5, 5, 4, 4, 5,
        4, 3, 4, 4, 2, 5, 5, 3, 5, 4, 4, 5, 4, 5, 4, 5, 5, 4, 4, 4, 5, 5, 3, 4,
        4, 2, 2, 5, 5, 5, 5, 3, 5, 3, 4, 5, 5, 4, 4, 5, 5, 4, 4, 3, 4, 4, 5, 4,
        3, 3, 4, 4, 5, 5, 4, 5, 3, 4, 2, 5, 4, 5, 4, 5, 5, 4, 5, 3, 3, 5, 4, 4,
        3, 5, 5, 4, 3, 2, 3, 4, 4, 5, 5, 5, 5, 3, 1, 5, 4, 4, 1, 5, 3, 3, 4, 5,
        4, 4, 4, 5, 5, 5, 4, 4, 4, 4, 3, 4, 4, 4, 5, 5, 2, 4, 4, 3, 3, 4, 4, 4])


In [7]:
loss_dict["logit_m_list"][0].shape

torch.Size([770, 1029])

In [2]:
# ckpts = checkpoint_utils.load_model_ensemble_and_task(["/homes/nw003/workspace/stage0/MERT/models/MERT-v1-95M_fairseq.pt"])
ckpt_path = "/data/scratch/acw713/mert/results/25-06-23-logistic-float16-continuous/ckpt_MERT_RVQ-VAE_CQT/MERT_RVQ-VAE_CQT_95M/checkpoint_last.pt"
models, cfg, task = checkpoint_utils.load_model_ensemble_and_task([ckpt_path])

/data/home/acw713/workspace/mert/MERT/src/fairseq/fairseq/checkpoint_utils.py:340: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(f, map_location=torch.dev

CQT kernels created, time used = 1.5077 seconds


In [3]:
model = models[0]
model.eval()

MERTModel(
  (feature_extractor): ConvFeatureExtractionModel(
    (conv_layers): ModuleList(
      (0): Sequential(
        (0): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (1): Dropout(p=0.0, inplace=False)
        (2): Fp32GroupNorm(512, 512, eps=1e-05, affine=True)
        (3): GELU(approximate='none')
      )
      (1-4): 4 x Sequential(
        (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (1): Dropout(p=0.0, inplace=False)
        (2): GELU(approximate='none')
      )
      (5-6): 2 x Sequential(
        (0): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        (1): Dropout(p=0.0, inplace=False)
        (2): GELU(approximate='none')
      )
    )
  )
  (post_extract_proj): Linear(in_features=512, out_features=768, bias=True)
  (dropout_input): Dropout(p=0.1, inplace=False)
  (dropout_features): Dropout(p=0.1, inplace=False)
  (encoder): TransformerEncoder(
    (pos_conv): Sequential(
      (0): Conv1d(768, 768, kern

In [4]:
audio, sr = torchaudio.load("/data/scratch/acw713/datasets/fma4all/wav/107036.wav")
audio = torchaudio.functional.resample(audio, orig_freq=sr, new_freq=24000)[:, :120000*5]
sources = torch.chunk(audio, 5, dim=1)
sources = torch.cat(sources, dim=0)

In [5]:
x = model.extract_features(sources, ret_layer=False)
cqt_pred_m = model.encoder_cqt_model(x)
source_cqt = model.encoder_cqt_model.compute_cqt(sources)

: 

In [ ]:
cqt_model = model_cqt_pred(768, 336, 24000, 75, logistic=True)
cqt_pred_m = cqt_model(x[mask_indices], forward_type="masked_logistic_output")
source_cqt.shape

In [ ]:
source_cqt = source_cqt[:mask_indices.shape[0], :mask_indices.shape[1]]
loss = cqt_model.criterion(cqt_pred_m, source_cqt[mask_indices])
loss

In [ ]:
data = cqt_model.target_scaler(source_cqt[mask_indices])
data.min(), data.max()

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(cqt_pred_m[1].detach().numpy().T, origin='lower', aspect='auto', vmin=0, vmax=2)

In [ ]:
plt.imshow(source_cqt[1].detach().numpy().T, origin='lower', aspect='auto', vmin=0, vmax=2)

In [ ]:
gtzan = GTZAN("/import/c4dm-05/nw003/datasets/gtzan", segment_length=5, sample_rate=24000, duration=30)

In [ ]:
gtzan_dataset = gtzan.dataset(transform="AudioFeature", preload="./test_data", encoder=model, trainset_size=10)

In [ ]:
gtzan_dataset.setup("fit", )

In [ ]:
train_iter = iter(gtzan_dataset.val_dataloader())

In [ ]:
batch = next(train_iter)

In [ ]:
batch[0].shape